In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import matplotlib.pyplot as plt
import numpy as np

# This is heavily based on the given solution for the class VAE exercise

In [2]:
import os
os.makedirs("results", exist_ok=True)

cuda2 = False # torch.cuda.is_available() check if a gpu is available
batch_size2 = 128
log_interval2 = 10
epochs2 = 2 # 10
latent_dim = 2

torch.manual_seed(1) # args.seed

device = torch.device("cuda" if cuda2 else "cpu") # args.cuda
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda2 else {} # args.cuda

# Get train and test data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size2, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size2, shuffle=True, **kwargs)

In [3]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD # -ELBO

def train(model, optimizer, epoch):
    model.train() # so that everything was gradients and we can do backprop and so on...
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad() # "reset" gradients to 0 for text iteration
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward() # calc gradients
        train_loss += loss.item()
        optimizer.step() # backpropagation

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(model, epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad(): # no_grad turns of gradients...
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [4]:
# Define VAE model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc1a = nn.Linear(400, 100)
        self.fc21 = nn.Linear(100, latent_dim) # 20 -> 2
        self.fc22 = nn.Linear(100, latent_dim) # 20 -> 2
        self.fc3 = nn.Linear(latent_dim, 100) # 20 -> 2
        self.fc3a = nn.Linear(100, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc1a(h1))
        mu, logvar = self.fc21(h2), self.fc22(h2)
        return torch.distributions.normal.Normal(mu, torch.exp(0.5*logvar)), mu, logvar

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        h4 = F.relu(self.fc3a(h3))
        return torch.sigmoid(self.fc4(h4))

    def forward(self, x):
        gauss, mu, logvar = self.encode(x.view(-1, 784))
        z = gauss.rsample()
        return self.decode(z), mu, logvar

In [5]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, epochs2 + 1):
    train(model, optimizer, epoch)
    test(model, epoch)
    with torch.no_grad():
        sample = torch.randn(64, latent_dim).to(device) # 20 -> 2
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png'
                  )

====> Epoch: 1 Average loss: 188.5651
====> Test set loss: 166.2209
====> Epoch: 2 Average loss: 162.5133
====> Test set loss: 159.2429


# Estimating model evidence

In [6]:
model.eval()
x0, _y0 = iter(test_loader).next()

In [53]:
# Returns batch of marginal likelihood estimates.
def importance_sampling(
    N, # number of samples to use for Monte Carlo estimator
    x, # batch of images
):
    # Importance sampling for VAE. Oswin PML script, pp. 65.
    # and Kingma and Welling. "An Introduction to Variational Autoencoders".
    # Section 2.6.
    with torch.no_grad():
        # Draw from proposal distribution q(z|x) (decoder)
        qz_x, mu, logvar = model.encode(x)
        z = qz_x.rsample((N,))
        
        # Kingma and Welling, Section 1.7.3 for binary data:
        #TODO; ours is not really binary, but neither is loss func?

        # Note that the posterior has diagonal covariance
        # by definition of the decoder in the VAE class above.
        # Compute log q(z|x) = N(z;mu, diag(var))
        log_qz_x = qz_x.log_prob(z).sum(-1)
        #                          ^since diagonal variance => independent RVs
        #                           q(z|x) = prod_i q(z_i|x)
        #                           log q(z|x) = sum_i log q(z_i|x)
        #                           See Kingma Section 2.5.
        # Compute log p(z) = log N(z;0,I)
        log_pz = torch.distributions.normal.Normal(
            loc=torch.zeros_like(mu),
            scale=torch.ones_like(logvar),
        ).log_prob(z).sum(-1)
        # Compute p(x|z) = sum_j log p(x_j|z) (by IID assumption)
        #                = sum_j log Bernoulli(x_j; p_j)
        #                  with p = decoder(z)
        #                = sum_j x_j log p_j + (1-x_j) * log(1-p_j)
        #                = -1*BCE of x and decoder(z)
        log_px_z = -1*F.binary_cross_entropy(
            model.decode(z),   # probabilities of Bernoulli R.V.s
            x.repeat((N,1,1)), # match decoder(z) shape
            reduction="none",
        ).sum(-1)
        # ^sum over x feature dimension

        # p(x) ~= 1/N sum_i^N p(x|z^(i))p(z^(i)) / q(z^(i)|x)
        px = torch.mean(log_px_z + log_pz - log_qz_x, 0)
        
        #print("z", z.shape)
        #print("log_qz_x", log_qz_x.shape)
        #print("log_pz  ", log_pz.shape)
        #print("log_px_z", log_px_z.shape)
        #print("px", px.shape)
        return px


importance_sampling(10, x0.view(-1, 784))[:3]

tensor([-165.8566,  -49.7395, -163.1648])

In [55]:
num_samples = 10
px_estimates = []
for (x, _y) in test_loader:
    x = x.view(-1, 784).to(device)
    px_estimates.append(importance_sampling(num_samples, x))

pX = torch.cat(px_estimates).mean()
print('p(x) ~= {:.4f}'.format(pX))

p(x) ~= -159.1983
